<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/test_sample_Distillation(Encoder_Decoder).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install sentencepiece

In [13]:
import torch
import torch.nn as nn
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [4]:
#############
# Settings
#############

teacher = 't5-small'
e = 3 # --> target encoder layers
d = 3 # --> target decoder layers
save_path = 'save_folder'

In [5]:
AutoTokenizer.from_pretrained(teacher).save_pretrained(save_path) # --> purely for convenience
teacher = AutoModelForSeq2SeqLM.from_pretrained(teacher).eval()

In [6]:
init_kwargs = teacher.config.to_diff_dict()
teacher_e, teacher_d = teacher.config.num_layers, teacher.config.num_decoder_layers
init_kwargs.update({'num_layers': e, 'num_decoder_layers': d}) # --> update configuration with target encoder & decoder layers

In [7]:
student_cfg = teacher.config_class(**init_kwargs)
student = AutoModelForSeq2SeqLM.from_config(student_cfg)
info = student.load_state_dict(teacher.state_dict(), strict=False)
assert info.missing_keys == [], info.missing_keys # --> every student should have teacher keys

In [8]:
LAYERS_TO_COPY = {
    6: {
        1: [0],
        2: [0, 5],
        3: [0, 2, 5],
        4: [0, 1, 3, 5],
        6: list(range(6)),
    }
}

def pick_layers_to_copy(n_student, n_teacher):
  val = LAYERS_TO_COPY[n_teacher][n_student]
  return val

In [9]:
if e is not None:
  e_layers_to_copy = pick_layers_to_copy(e, teacher_e)

if d is not None:
  d_layers_to_copy = pick_layers_to_copy(d, teacher_d)

In [14]:
def copy_layers(src_layers, dest_layers, layers_to_copy):
  layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
  dest_layers.load_state_dict(layers_to_copy.state_dict())

# For T5 this method has to be done
copy_layers(teacher.encoder.block, student.encoder.block, e_layers_to_copy)
copy_layers(teacher.decoder.block, student.decoder.block, d_layers_to_copy)

In [16]:
student.config.init_metadata = dict(
    teacher_type = teacher.config.model_type,
    copied_encoder_layers = e_layers_to_copy,
    copied_decoder_layers = d_layers_to_copy,
)

In [17]:
student.save_pretrained(save_path)